In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers, models, utils

### Define global varibles as needed

In [ ]:
%matplotlib inline
DATASET_DIR = 'dataset'
BATCH_SIZE = 10
DATASET_SIZE = 5000
ID_FROM = 100_000
ID_TO = 133_332
IMAGES_PEER_ID = 5
# IMAGE_SHAPE = (224, 224, 3)
IMAGE_SHAPE = (112, 112, 3)


## Its recomended to use other dataset more quality images than [DigiFace1M](https://github.com/microsoft/DigiFace1M) dataset

### Download first part of [DigiFace1M](https://github.com/microsoft/DigiFace1M) dataset.

In [ ]:
# ! wget https://facesyntheticspubwedata.blob.core.windows.net/wacv-2023/subjects_100000-133332_5_imgs.zip -o faces.zip
# ! mkdir dataset && cd dataset && unzip ../faces.zip && rm -rf ../faces.zip

### Prepare DATASET: Set (anchor, positive), (anchor, negative), and labels 1: positive, 0: negative

In [ ]:

X_anchor = []
X_verify = []

y_dataset: list[int] = []

np.random.seed(int(time.time()))

def get_rand_id() -> int :
  return np.random.randint(ID_FROM, ID_TO)

def get_rand_img() -> int :
  return np.random.randint(0, IMAGES_PEER_ID)

def load_rand_image_of_id(dir: int) -> np.array:
  # rand_img: int = get_rand_img()
  # path = os.path.join(DATASET_DIR, f'{dir}', f'{rand_img}.png')
  path = os.path.join(DATASET_DIR, f'{dir}', '0.png')
  # image = utils.load_img(path, color_mode='grayscale')
  image = utils.load_img(path)
  image = utils.img_to_array(image)
  image = utils.normalize(image)
  return image

def set_dataset() -> None:
  global X_dataset, y_dataset

  for i in range(DATASET_SIZE // 2):
    anchor_id = get_rand_id()

    anchor_img = load_rand_image_of_id(anchor_id)
    # pos_img = load_rand_image_of_id(anchor_id)
    negative_img = load_rand_image_of_id(get_rand_id())

    X_anchor.append(anchor_img)
    X_verify.append(anchor_img)
    X_anchor.append(anchor_img)
    X_verify.append(negative_img)

    y_dataset.append(1)
    y_dataset.append(0)

set_dataset()

X_anchor = np.asarray(X_anchor, dtype=np.float32)
X_verify = np.asarray(X_verify, dtype=np.float32)
y_dataset = np.asarray(y_dataset, dtype=np.float32)

train_size = (DATASET_SIZE * 4) // 5
X_anchor_train, X_anchor_test = X_anchor[:train_size, :, :, :], X_anchor[train_size:, :, :, :]
X_verify_train, X_verify_test = X_verify[:train_size, :, :, :], X_verify[train_size:, :, :, :]
y_train, y_test = y_dataset[:train_size], y_dataset[train_size:]


## Define the model and training process

###define the input convolutional network

In [ ]:
def make_embeding() -> models.Model:

  input = layers.Input(shape=IMAGE_SHAPE, batch_size=BATCH_SIZE, name='input_image')

  layer = layers.Conv2D(filters=64, kernel_size=5, strides=(1, 1), padding='same', activation='relu', name='conv1')(input)
  layer = layers.MaxPooling2D(pool_size=(2, 2), name='maxpooling1')(layer)
  layer = layers.BatchNormalization(name='batchnorm1')(layer)

  layer = layers.Conv2D(filters=64, kernel_size=1, strides=(1, 1), padding='same', activation='relu', name='conv2a')(layer)
  layer = layers.Conv2D(filters=96, kernel_size=3, strides=(1, 1), padding='same', activation='relu', name='conv2b')(layer)
  layer = layers.MaxPooling2D(pool_size=(2, 2), name='maxpooling2')(layer)
  layer = layers.BatchNormalization(name='batchnorm2')(layer)

  layer = layers.Conv2D(filters=96, kernel_size=1, strides=(1, 1), padding='same', activation='relu', name='conv3a')(layer)
  layer = layers.Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same', activation='relu', name='conv3b')(layer)
  layer = layers.MaxPooling2D(pool_size=(2, 2), name='maxpooling3')(layer)

  layer = layers.Conv2D(filters=128, kernel_size=1, strides=(1, 1), padding='same', activation='relu', name='conv4a')(layer)
  layer = layers.Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same', activation='relu', name='conv4b')(layer)

  layer = layers.Conv2D(filters=256, kernel_size=1, strides=(1, 1), padding='same', activation='relu', name='conv5a')(layer)
  layer = layers.Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same', activation='relu', name='conv5b')(layer)

  layer = layers.Conv2D(filters=128, kernel_size=1, strides=(1, 1), padding='same', activation='relu', name='conv6a')(layer)
  layer = layers.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same', activation='relu', name='conv6b')(layer)
  layer = layers.MaxPooling2D(pool_size=(2, 2), name='maxpooling6')(layer)

  layer = layers.Conv2D(filters=64, kernel_size=1, strides=(1, 1), padding='same', activation='relu', name='conv7a')(layer)
  layer = layers.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same', activation='relu', name='conv7b')(layer)
  layer = layers.MaxPooling2D(pool_size=(2, 2), name='maxpooling7')(layer)

  layer = layers.Flatten(name='flatten')(layer)
  output = layers.Dense(units=1024, activation='relu', name='dense')(layer)

  return models.Model(inputs=input, outputs=output, name='embeding_model')


In [ ]:
# distance layer
Distance = layers.Lambda(lambda pair: tf.abs(pair[0] - pair[1]), name='distance')

In [ ]:
def make_siamese_model() -> models.Model:

  anchor_img = layers.Input(shape=IMAGE_SHAPE, batch_size=BATCH_SIZE, name='anchor_img')
  verification_img = layers.Input(shape=IMAGE_SHAPE, batch_size=BATCH_SIZE, name='verification_img')

  embeding = make_embeding()

  anchor_embeding, verification_embeding = embeding(anchor_img), embeding(verification_img)

  distance = Distance([anchor_embeding, verification_embeding])

  dense = layers.Dense(units=128, activation='relu', name='dense_distance')(distance)

  output = layers.Dense(units=1, activation='sigmoid', name='classifier')(dense)

  return models.Model(inputs=[anchor_img, verification_img], outputs=output, name='faceid_model')


In [ ]:
model = make_siamese_model()

In [ ]:
# # model.summary(expand_nested=True, positions=[0.4, 0.65, 0.75, 1.])
# utils.plot_model(
#       model,
#       to_file="faceid_model.png",
#       show_shapes=True,
#       show_layer_names=True,
#       # rankdir="LR",
#       expand_nested=True,
#       show_layer_activations=True
# )


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=[X_anchor_train, X_verify_train],y=y_train, batch_size=BATCH_SIZE, epochs=10)

In [ ]:
model.evaluate(x=[X_anchor_test, X_verify_test], y=y_test, batch_size=BATCH_SIZE)

In [ ]:
model.save('saved_model.pb')